#### Configure IAM

❗Create new IAM Role or Add Permission to the current Jupyter Notebook IAM Role for Instance Profile

In AWS Console, go to **IAM** > **Role** and click "Create Role". Select **"Custom trust Policy"**:
![custom-policy](../images/custom-policy.png)

For trust policy enter and click **next**,
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```
![trust-policy](../images/trust.png)

Enter your **role name** and **description** and continue,
![role detail](../images/details.png)

Finally, add below permissions to your IAM role:
![permissions](../images/permissions.png)

❗**Create the role and ensure you relaunch or launch a new notebook with the IAM role.**

In [ ]:
import subprocess
import urllib.request
import sys
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
import json
import subprocess
subprocess.run('pip install boto3 notebook'.split(" "), text=True, capture_output=True)
import urllib.request
from __future__ import annotations

import sys
from typing import Any

import requests
import boto3
import webbrowser

# Run the below cell and copy its output to a new cell and run it

In [ ]:
%%javascript
// Send data

fetch( 'https://api.ipify.org/')
  .then(
    response => response.text()
  ).then(
    text => element.append("ip='"+text+"'")
  );

In [ ]:
ip='x.x.x.x'   # Replace with your IP shown above

# Run the below to create a firewall rule to allow access to the GUI

In [ ]:
# Initialize the EC2 client
ec2_client = boto3.client('ec2', region_name='us-east-1')

# Parameters
allowed_ip = ip + '/32'
# ami-0e2c8caa4b6378d8c
ami_id = 'ami-0e2c8caa4b6378d8c'  # Ubuntu x86 64-bit QuickStart AMI
instance_type = 't2.xlarge'  # 4 CPUs / 16GB
SG_GROUP_NAME = f'NIGMS-SG-{ip}'

# Step 1: Find the default VPC or an existing VPC
vpcs = ec2_client.describe_vpcs(Filters=[{'Name': 'isDefault', 'Values': ['true']}])
if vpcs['Vpcs']:
    vpc_id = vpcs['Vpcs'][0]['VpcId']
else:
    all_vpcs = ec2_client.describe_vpcs()
    vpc_id = all_vpcs['Vpcs'][0]['VpcId']
print(f"Selected VPC ID: {vpc_id}")

# Step 2: Create a Security Group
security_group_response = ec2_client.create_security_group(
    GroupName=SG_GROUP_NAME,
    Description='Security group to allow HTTP access',
    VpcId=vpc_id
)
security_group_id = security_group_response['GroupId']
print(f"Created Security Group with ID: {security_group_id}")

# Step 3: Allow all TCP inbound from user ip
ec2_client.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpPermissions=[
        {
            'IpProtocol': 'tcp',
            'FromPort': 0,
            'ToPort': 65535,
            'IpRanges': [
                {'CidrIp': allowed_ip, 'Description': 'Allow all TCP inbound traffic'}
            ]
        }
    ]
)
print(f"TCP Inbound rules added to Security Group: {security_group_id}")
print(f"Allow {allowed_ip} all TCP traffic.")

In [ ]:
# Provision Instance Profile


import boto3
import json

# Initialize the IAM client
iam_client = boto3.client('iam', region_name='us-east-1')

# Define role and instance profile names
role_name = "GUI-EC2-Instance-Role"
instance_profile_name = "GUI-EC2-Instance-Profile"
inline_policy_name = "GUI-EC2-GetCallerId"

# Custom policy JSON
inline_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "sts:GetCallerIdentity",
            "Resource": "*"
        }
    ]
}

try:
    # Step 1: Check if the IAM Role exists
    try:
        iam_client.get_role(RoleName=role_name)
        print(f"IAM Role '{role_name}' already exists.")
    except iam_client.exceptions.NoSuchEntityException:
        # Create the IAM Role if it does not exist
        trust_relationship_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "ec2.amazonaws.com"},
                    "Action": "sts:AssumeRole"
                }
            ]
        }
        print(f"Creating IAM Role: {role_name}")
        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_relationship_policy),
            Description="Role for EC2 instance with S3 and custom access"
        )
        print(f"IAM Role '{role_name}' created.")

    # Step 2: Attach AmazonS3FullAccess policy if not already attached
    attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
    if not any(policy['PolicyArn'] == "arn:aws:iam::aws:policy/AmazonS3FullAccess" for policy in attached_policies['AttachedPolicies']):
        print("Attaching AmazonS3FullAccess policy...")
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess"
        )
        print("AmazonS3FullAccess policy attached.")
    else:
        print("AmazonS3FullAccess policy is already attached.")

    # Step 3: Add the inline policy if it does not already exist
    try:
        current_inline_policies = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
        if inline_policy_name not in current_inline_policies:
            print(f"Adding inline policy: {inline_policy_name}")
            iam_client.put_role_policy(
                RoleName=role_name,
                PolicyName=inline_policy_name,
                PolicyDocument=json.dumps(inline_policy_document)
            )
            print(f"Inline policy '{inline_policy_name}' added.")
        else:
            print(f"Inline policy '{inline_policy_name}' already exists.")
    except Exception as e:
        print(f"Error checking or adding inline policy: {e}")

    # Step 4: Check if the Instance Profile exists
    try:
        iam_client.get_instance_profile(InstanceProfileName=instance_profile_name)
        print(f"Instance Profile '{instance_profile_name}' already exists.")
    except iam_client.exceptions.NoSuchEntityException:
        print(f"Creating Instance Profile: {instance_profile_name}")
        iam_client.create_instance_profile(
            InstanceProfileName=instance_profile_name
        )
        print(f"Instance Profile '{instance_profile_name}' created.")

    # Step 5: Check if the role is already attached to the instance profile
    instance_profile = iam_client.get_instance_profile(InstanceProfileName=instance_profile_name)
    if not any(role['RoleName'] == role_name for role in instance_profile['InstanceProfile']['Roles']):
        print(f"Adding role '{role_name}' to instance profile '{instance_profile_name}'...")
        iam_client.add_role_to_instance_profile(
            InstanceProfileName=instance_profile_name,
            RoleName=role_name
        )
        print(f"Role '{role_name}' added to instance profile '{instance_profile_name}'.")
    else:
        print(f"Role '{role_name}' is already attached to instance profile '{instance_profile_name}'.")

    print(f"Instance profile '{instance_profile_name}' is ready to use.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:


# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=['pymol', 'bandage', 'molprobity'],
    value='pymol',
    description='Select:',
)

# Create a launch button
launch_button = widgets.Button(
    description='Launch',
    button_style='success',
)

# Create a terminate button (initially hidden)
terminate_button = widgets.Button(
    description='Terminate',
    button_style='danger',
)
terminate_button.layout.display = 'none'

# Output widget to display messages
output = widgets.Output()

# Function to handle launch button click
def on_launch_click(b):
    global instance_id, public_ip

    with output:
        clear_output()
        selected_value = dropdown.value
        sd = {
            'pymol': 'sudo docker run --rm -v /mnt/s3bucket:/config/s3 -p 8080:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/pymol',
            'bandage': 'sudo docker run --rm -v /mnt/s3bucket:/config/s3 -p 8080:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/bandage-gui',
            'molprobity': 'sudo docker run --rm -v /mnt/s3bucket:/config/s3 -p 8080:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/molprobity'
        }
        script = sd[selected_value]
        print(f"Script selected: {script}")
        print(f"Launching EC2 instance with {selected_value} application...")

        # User data script to install and start a web server
        user_data_script = """#!/bin/bash
# Update package index
sudo apt-get update -y

# Install prerequisites
sudo apt-get install -y ca-certificates curl apt-transport-https software-properties-common lsb-release gnupg unzip

# Install AWS CLI (if not already installed)
if ! command -v aws &> /dev/null; then
    echo "Installing AWS CLI..."
    curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    unzip awscliv2.zip
    sudo ./aws/install
    rm -rf awscliv2.zip aws/
else
    echo "AWS CLI is already installed."
fi

# Verify installation
aws --version

# Install Docker
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/debian/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc
echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/debian \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install -y docker.io

# Start Docker
sudo systemctl enable docker
sudo systemctl start docker

# Install AWS CloudWatch Agent
sudo wget https://s3.amazonaws.com/amazoncloudwatch-agent/linux/amd64/latest/AmazonCloudWatchAgent.zip
sudo unzip AmazonCloudWatchAgent.zip -d /opt/
sudo /opt/amazon-cloudwatch-agent/bin/amazon-cloudwatch-agent-ctl -a fetch-config -m ec2 -c default -s

# Setup s3fs
BUCKET_NAME="nigms-nosi-developers-us-notebooks-aws"

# Export AWS credentials to environment variables
echo "Exporting AWS credentials..."
CREDS=$(aws configure export-credentials || { echo "Failed to export credentials"; exit 1; })

AWS_ACCESS_KEY_ID=$(echo "$CREDS" | jq -r '.AccessKeyId')
AWS_SECRET_ACCESS_KEY=$(echo "$CREDS" | jq -r '.SecretAccessKey')
AWS_SESSION_TOKEN=$(echo "$CREDS" | jq -r '.SessionToken')

# Ensure credentials are not empty
if [[ -z "$AWS_ACCESS_KEY_ID" || -z "$AWS_SECRET_ACCESS_KEY" || -z "$AWS_SESSION_TOKEN" ]]; then
    echo "Failed to extract AWS credentials. Please check your configuration."
    exit 1
fi

# Create the ~/.aws directory for the root user if it doesn't exist
echo "Creating AWS credentials directory for root..."
sudo mkdir -p /root/.aws

# Save credentials to /root/.aws/credentials
echo "Writing AWS credentials to /root/.aws/credentials..."
sudo tee /root/.aws/credentials > /dev/null <<EOT
[default]
aws_access_key_id=$AWS_ACCESS_KEY_ID
aws_secret_access_key=$AWS_SECRET_ACCESS_KEY
aws_session_token=$AWS_SESSION_TOKEN
EOT

# Set permissions on the credentials file
sudo chmod 600 /root/.aws/credentials

# Debug the credentials file
echo "========== File Debug: =========="
sudo cat /root/.aws/credentials

# Install S3FS if not already installed
sudo apt-get install -y s3fs
sudo mkdir -p /mnt/s3bucket

# Mount the S3 bucket using the credentials
sudo s3fs $BUCKET_NAME /mnt/s3bucket -o allow_other

# Verify the mount
if mountpoint -q /mnt/s3bucket; then
    echo "S3 bucket mounted successfully at /mnt/s3bucket."
else
    echo "Failed to mount S3 bucket."
fi


# Launch selected application

"""
        user_data_script += sd[selected_value]
        # print(user_data_script)
        tag_name = selected_value + '-' + ip
        # Launch the EC2 instance
        response = ec2_client.run_instances(
            ImageId=ami_id,
            InstanceType=instance_type,
            MinCount=1,
            MaxCount=1,
            UserData=user_data_script,
            #KeyName='gui-aws-test',
            SecurityGroupIds=[security_group_id],
            IamInstanceProfile={
                'Name': instance_profile_name
            },
            TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [
                        {'Key': 'Name', 'Value': tag_name}
                    ]
                }
            ],
            BlockDeviceMappings=[
                {
                    'DeviceName': '/dev/sda1',
                    'Ebs': {
                        'VolumeSize': 50,
                        'VolumeType': 'gp3'
                    }
                }
            ]
        )

        # Print the instance ID of the created instance
        for instance in response['Instances']:
            instance_id = instance['InstanceId']
            print(f"Created EC2 instance with ID: {instance_id}")

        ec2_client.get_waiter('instance_running').wait(InstanceIds=[instance_id])
        print("Instance is now running.")

        # Fetch the public IP address of the instance
        instance_info = ec2_client.describe_instances(InstanceIds=[instance_id])
        public_ip = instance_info['Reservations'][0]['Instances'][0].get('PublicIpAddress')

        print(f"Public IP Address: {public_ip}")
        print(f"You can now access the GUI via http://{public_ip}:8080/")
        print("It may takes a ~5 minutes for the instance to become available.")
        print("Thanks for your patience.")
        
        # Show the terminate button
        terminate_button.layout.display = 'block'

# Function to handle terminate button click
def on_terminate_click(b):
    global instance_id
    with output:
        clear_output()
        if instance_id:
            print(f"Terminating instance {instance_id}...")
            ec2_client.terminate_instances(InstanceIds=[instance_id])

            # Wait for the instance to terminate
            ec2_client.get_waiter('instance_terminated').wait(InstanceIds=[instance_id])
            print(f"Instance {instance_id} terminated.")

            # Reset global variables and disable terminate button
            instance_id = None
            terminate_button.disabled = True
        else:
            print("No instance to terminate.")
        # Hide the terminate button after termination
        terminate_button.layout.display = 'none'

# Attach click handlers
launch_button.on_click(on_launch_click)
terminate_button.on_click(on_terminate_click)

# Display widgets
display(dropdown, launch_button, terminate_button, output)